# Invoking models from Amazon Bedrock

<div class="alert alert-block alert-info">
⚠️ <b>Important:</b> Make sure you've run the <code>0-setup/1-required-dependencies.ipynb</code> notebook in this repository before proceeding.
</div>

In [ ]:
import boto3
import json

bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=boto3.Session().region_name
)

def invoke_model(model_id, model_input):
    body = json.dumps(model_input)
    
    response = bedrock.invoke_model(
        modelId=model_id,
        body=body
    )
    
    response_body = json.loads(response['body'].read())
    return response_body

In [ ]:
%%time
deepseek_prompt = "What are the key ingredients for pizza?"
deepseek_input = {
    "prompt": f"<｜begin_of_sentence｜><｜User｜>{deepseek_prompt}<｜Assistant｜><think>\n",
    "temperature": 0.5,
    "max_tokens": 512,
}
deepseek_response = invoke_model('us.deepseek.r1-v1:0', deepseek_input)
deepseek_response

In [ ]:
%%time
mistral_prompt = "What are the key ingredients for pizza?"
mistral_input = {
    "prompt": f"<s>[INST] {mistral_prompt} [/INST]",
    "temperature": 0.5,
    "max_tokens": 512,
}
mistral_response = invoke_model('mistral.mistral-large-2402-v1:0', mistral_input)
mistral_response

## Using the Converse API

In [ ]:
%%time
nova_prompt = "What are the key ingredients for pizza?"
messages = [{"role": "user", "content": [{"text":nova_prompt}]}]
nova_response = bedrock.converse(
    modelId="us.amazon.nova-pro-v1:0",
    messages=messages,
    system=[{"text":"You are a helpful assistant."}],
    inferenceConfig={
        "maxTokens": 512,
        "temperature": 0.5,
    }
)
nova_response

## Using LiteLLM

In [ ]:
from litellm import completion

In [ ]:
%%time

response = completion(
    model="bedrock/us.amazon.nova-pro-v1:0",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What are the key ingredients for pizza?"}
    ],
    max_tokens=512,
    temperature=0.5,
)
response